### **一、加载常用库**

In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import gc
import statsmodels.api as sm
pd.set_option('display.max_columns', None)

C:\Users\Administrator\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Administrator\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


### **二、计算风格因子Beta和残差标准差resid_std**

In [2]:
#半衰期加权函数
def Sma_half(df,h):
    #sma均值
    #df1 = df.ewm(alpha=m/n).mean()
    alpha = 1-np.exp(np.log(0.5)/h)
    return df.ewm(alpha = alpha , adjust=False).mean()

#滚动回归
def OLS_params(df,window):
    result = pd.DataFrame()
    listcode = df.code.unique()
    for code in listcode:
        df_sub = df[df.code == code].reset_index(drop = True)
        for row in range(window, len(df_sub)+1):
            y = df_sub.loc[row-window:row,['return_half']]
            x = df_sub.loc[row-window:row,['Retindex_half']]
            #X = sm.add_constant(x)
            model = sm.OLS(y,x).fit()
            df_sub.loc[row-1,['Beta']] = model.params[0]
            df_sub.loc[row-1,['resid_std']] = model.resid.std()
        result = pd.concat([result,df_sub])
    
    return result

In [3]:
#导入股票和指数交易数据
stktrd = pd.read_hdf('alldata.h5',key='stktrd')

In [4]:
#提取出需要的列
stktrd_getbeta = stktrd.loc[:,['code','date','return','Retindex']]

In [5]:
#计算半衰指数加权股票收益率
stktrd_getbeta['return_half'] = stktrd_getbeta.groupby(['code'])['return'].apply(lambda x: Sma_half(x,60))

In [6]:
#计算半衰指数加权沪深300收益率
stktrd_getbeta['Retindex_half'] = stktrd_getbeta.groupby(['code'])['Retindex'].apply(lambda x: Sma_half(x,60))

In [7]:
ols_result = OLS_params(stktrd_getbeta,250)

In [14]:
ols_result.reset_index(drop = True, inplace = True)

In [16]:
ols_result.to_hdf('ols_result.h5', key = 'ols_result')

In [17]:
ols_result

,code,date,return,Retindex,return_half,Retindex_half,Beta,resid_std
0,1,2010-01-04,-0.027082,-0.011314,-0.027082,-0.011314,NaN,NaN
1,1,2010-01-05,-0.017292,0.008149,-0.026970,-0.011090,NaN,NaN
2,1,2010-01-06,-0.017167,-0.006260,-0.026857,-0.011035,NaN,NaN
3,1,2010-01-07,-0.010917,-0.019841,-0.026674,-0.011136,NaN,NaN
4,1,2010-01-08,-0.002208,0.002499,-0.026393,-0.010979,NaN,NaN
...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,-0.026646,0.001553,-0.000022,-0.000290,1.668251,0.000730
8672605,900957,2022-01-25,-0.016103,-0.022622,-0.000207,-0.000547,1.697025,0.000720
8672606,900957,2022-01-26,0.004910,0.007237,-0.000148,-0.000457,1.713335,0.000714
8672607,900957,2022-01-27,-0.003257,-0.019615,-0.000184,-0.000677,1.726680,0.000709
